In [19]:
import urllib.request as urllib
from bs4 import BeautifulSoup
import re
from contextlib import closing
import sqlite3
import time
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
options = Options()
options.add_argument('--headless')
 
browser = webdriver.Chrome(chrome_options=options)

c:\users\icech\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: use options instead of chrome_options
  del sys.path[0]


In [43]:
def get_db_conn(name):
    '''
    sql文を何度も書きたくなかったので書いた
    '''
    conn=sqlite3.connect(name)
    c=conn.cursor()
    return(conn,c)

def get_category(product_url):
    browser.get(product_url)
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    category=soup.select('.a-unordered-list')[0].text
    category=re.sub('›','',category)
    category=re.sub('\n','',category)
    category=category.strip()
    category=category.split()
    
    return(category)

def get_each_product_info():
    '''
    レビュー詳細URLから必要な情報を手に入れるコード
    SQLに紐づけて必要な部分を後から埋めるでもいいかもしれない。
    categoryの数が変動するので注意
    '''
    url2='https://www.amazon.co.jp/gp/customer-reviews/R1JYHF8TMEKK3J?ref=pf_vv_at_pdctrvw_srp'
    browser.get(url2)
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    
    review_title=soup.select('.review-title')[0].text
    star=float(re.sub('5つ星のうち','',soup.select('.a-icon-alt')[0].text))
    text=soup.select('.review-text')[0].text
    
    product_url='https://www.amazon.co.jp'+soup.select('.a-text-ellipsis > a')[0].get('href')
    category=get_category(product_url)
    
    info_list=[]
    info_list.extend([star,review_title,text])
    info_list.extend(category)
    info_list.extend(['']*(13-len(info_list)))
    
    return(tuple(info_list))

In [44]:
get_each_product_info()

(4.0,
 '以前はシート状のつめみがき&艶出しをたまにしようしていました。この製品は磨き方だけで磨きとツヤ出し両方できるので楽ですし確実に早いです。またしっかりとツヤがでてくれて長期間続きます。男性で少し爪をきれいにしておきたいという人にもいいのではないでしょうか。',
 '簡単にピカピカになります',
 'ビューティー',
 'ネイル',
 'ネイル道具・ケアツール',
 '爪やすり',
 '',
 '',
 '',
 '',
 '',
 '')

In [32]:
low_dbname='1107_low_rate_bakawall_each_prodcuts.db'
high_dbname='1107_high_rate_bakawall_each_prodcuts.db'
Hconn,Hc=get_db_conn(high_dbname)
Lconn,Lc=get_db_conn(low_dbname)



In [14]:
Hc.execute('''select * from sqlite_master''')
Hc.execute('''create tabel product_text (id integer primary key,username,baka_star,each_id,pd_url,pd_star,pd_title,pd_text)''')
for i in range(1,11):
    Hc.execute(f'alter table product_text add column category{i}')

Hc.exectute('insert into product_text(username,baka_star,each_id,pd_url) select name,id,rate,url from products')




Lc.execute('''select * from sqlite_master''')
Lc.execute('''create tabel product_text (id integer primary key,username,baka_star,each_id,pd_url,pd_star,pd_title,pd_text)''')
for i in range(1,11):
    Lc.execute(f'alter table product_text add column category{i}')

Lc.exectute('insert into product_text(username,baka_star,each_id,pd_url) select name,id,rate,url from products')

# for h in Hc:
#     print(h)

('table', 'products', 'products', 2, 'CREATE TABLE products (id,name,rate,url,review,text, category, product_name)')


In [46]:
#idとrateが逆になっている
# Hc.execute('alter table products add column category')
# Hc.execute('alter table products add column product_name')


Hc.execute('''select distinct * from products where rate<101 ''')
j=0
for h in Hc:
    print(h)
    j+=1
    if j==10:
        break
    


(5.0, 'ぽんちゃん', 1, 'https://www.amazon.co.jp/gp/customer-reviews/R1JYHF8TMEKK3J?ref=pf_vv_at_pdctrvw_srp', None, None, None, None)
(5.0, 'ぽんちゃん', 2, 'https://www.amazon.co.jp/gp/customer-reviews/R2MHFHH3DKVKC0?ref=pf_vv_at_pdctrvw_srp', None, None, None, None)
(5.0, 'ぽんちゃん', 3, 'https://www.amazon.co.jp/gp/customer-reviews/R7D1ABOQPG1VQ?ref=pf_vv_at_pdctrvw_srp', None, None, None, None)
(5.0, 'ぽんちゃん', 4, 'https://www.amazon.co.jp/gp/customer-reviews/R3MZVX5CX8ILJ?ref=pf_vv_at_pdctrvw_srp', None, None, None, None)
(5.0, 'ぽんちゃん', 5, 'https://www.amazon.co.jp/gp/customer-reviews/R2CQ2PTLNGEV6O?ref=pf_vv_at_pdctrvw_srp', None, None, None, None)
(5.0, 'ぽんちゃん', 6, 'https://www.amazon.co.jp/gp/customer-reviews/RS35YP90IBJKK?ref=pf_vv_at_pdctrvw_srp', None, None, None, None)
(5.0, 'ぽんちゃん', 7, 'https://www.amazon.co.jp/gp/customer-reviews/R1Q9FT2VKIQ16J?ref=pf_vv_at_pdctrvw_srp', None, None, None, None)
(5.0, 'ぽんちゃん', 8, 'https://www.amazon.co.jp/gp/customer-reviews/R2B2H0N9CGAV85?ref=pf_vv_at_p